# Kontur.fokus ultimate beneficiary scraper

Aim here is to feed API queries with tons of INN (taxpayer numbers) to Kontur fokus API, to extract all info about shareholders, CEOs and affiliated companies from it.

### Section 1: Setup

In [3]:
import requests
import re
from requests.adapters import HTTPAdapter
from urllib.parse import urlparse, parse_qs
from urllib3.util import Retry
from bs4 import BeautifulSoup,SoupStrainer
from datetime import date
from dataclasses import dataclass
from tqdm import tqdm
from dateutil import parser
from threading import Thread
import pandas as pd
from datetime import datetime, date, timedelta
import logging
import http.client
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed
import math
from os import walk
import json
import numpy as np
import csv
from urllib import response
from webbrowser import get


In [2]:
with open(r"C:\Users\majda\Desktop\inn.csv") as f:
    sample = [str(s) for line in f.readlines() for s in line[:-1].split(',')]
    print(sample)

['6680000422', '6680000447', '6680001345', '6680001722', '6680002370', '6680005928', '6680007731', '6681000834', '6681005568', '6681005960', '6681007572', '6682001414', '6682004334', '6682007342', '6683000011', '6683003870', '6683005482', '6683005789', '6683007257', '6683010394', '6683010806']


In [8]:

def parseOwners(data, propertyName, ownerType, i): 
    result = []

    ## jesli data nie zawiera propertyName, to wrzuca pojedynczy wpis 
    if data[propertyName] == []:
        result.append({'id':i, 'bene':'N/A', 'shares':'N/A','ownerType': ownerType})

    ## jesli zawiera to iteruje po nich
    else:
        for owner in data[propertyName]:

            ## to pobiera nazwe jesli istnieje. 
            ## dla 'beneficialOwnersFL' jest w 'fio'
            ## dla 'beneficialOwnersOther' jest w 'fullName'
            bene = "";
            if 'fio' in owner:
                bene = owner['fio']
            elif 'fullName' in owner:
                bene = owner['fullName']

            # To pobiera shares jesli istnieja 
            shares= 'N/A'
            if 'share' in owner:
                shares = owner['share']

            result.append({'id':i, 'bene':bene, 'shares':shares,'ownerType': ownerType}) 

    return result

def getData():
    output=[]
    id=[]
    est=[]
    cap=[]
    bene=[]
    shares=[]
    historical=[]
    
    inn=['6608005109']
    
    for i in inn:
        print(i)
        temp=f"https://focus-api.kontur.ru/api3/beneficialOwners?inn={i}&key=4bcd0b82f0b0ea748eb8a984e47a0a6a48426667"
        response = requests.get(temp)
        id=response.json()[0]['inn']
        
        try:
            cap=response.json()[0]['statedCapital']['sum']
            est=response.json()[0]['statedCapital']['date']
        except:
            est=("")
            cap=("")
                    
        owners = response.json()[0]['beneficialOwners']
        Howners= response.json()[0]['historicalBeneficialOwners']
        
        output += parseOwners(owners, 'beneficialOwnersFL', 'FL', i)
        output += parseOwners(owners, 'beneficialOwnersForeign', 'FGN', i)
        output += parseOwners(owners, 'beneficialOwnersUL', 'UL', i)
        output += parseOwners(owners, 'beneficialOwnersOther', 'Other', i)

        output += parseOwners(Howners, 'beneficialOwnersFL', 'hFL', i)
        output += parseOwners(Howners, 'beneficialOwnersForeign', 'hFGN', i)
        output += parseOwners(Howners, 'beneficialOwnersUL', 'hUL', i)
        output += parseOwners(Howners, 'beneficialOwnersOther', 'hOther', i)

    
    return output

In [9]:
data=getData()

print(len(data))
print(data) 


6608005109
13
[{'id': '6608005109', 'bene': 'Чернецкий Станислав Аркадьевич', 'shares': 29.1299998760223, 'ownerType': 'FL'}, {'id': '6608005109', 'bene': 'Брижицкий Алексей Николаевич', 'shares': 10.6100000441074, 'ownerType': 'FL'}, {'id': '6608005109', 'bene': 'Котова Дарья Ивановна', 'shares': 9.97999981045723, 'ownerType': 'FL'}, {'id': '6608005109', 'bene': 'Баженов Александр Анатольевич', 'shares': 9.71999987959862, 'ownerType': 'FL'}, {'id': '6608005109', 'bene': 'Клименко Сергей Викторович', 'shares': 9.71999987959862, 'ownerType': 'FL'}, {'id': '6608005109', 'bene': 'N/A', 'shares': 'N/A', 'ownerType': 'FGN'}, {'id': '6608005109', 'bene': 'N/A', 'shares': 'N/A', 'ownerType': 'UL'}, {'id': '6608005109', 'bene': 'Администрация Города Екатеринбурга', 'shares': 29.2899996042252, 'ownerType': 'Other'}, {'id': '6608005109', 'bene': 'Галустян Юрий Иванович', 'shares': 'N/A', 'ownerType': 'hFL'}, {'id': '6608005109', 'bene': 'Гендина Александра Юрьевна', 'shares': 'N/A', 'ownerType':

In [32]:
df=pd.DataFrame(data,columns=['id','est', 'ownerType', 'bene','historical'])
df.to_csv("APItest!")